# O intuito desse notebook e recuperar todas as imagens disponiveis  sobre pneumonia e covid e disponibilizar em um banco de dados


## bibliotecas necessárias

In [3]:
!pip install imutils

Processing c:\users\wesley.neves\appdata\local\pip\cache\wheels\fc\9c\6d\1826267c72afa51b564c9c6e0f66abc806879338bc593a2270\imutils-0.5.3-py3-none-any.whl


In [1]:
from imutils import paths
import pydicom as dicom
import pandas as pd
import shutil
import cv2 
import os

##  diretorios target e source

In [2]:
def getTargetDiretorio(tipo):
    retorno = ""
    if(tipo =="MaquinaLocal"):
        retorno = r'D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto'

    if(tipo =="GoogleDrive"):
       retorno=r'/content/drive/My Drive/TCC Pos Graduação big data/TCC/Projeto/'
       
    
    return retorno

In [3]:
dir_target_diretorio_raiz = getTargetDiretorio("MaquinaLocal")
dir_target_diretorio_dataset =r"{0}/{1}".format(dir_target_diretorio_raiz,'dataset')
dir_target_diretorio_imagem_covid =r"{0}/{1}".format(dir_target_diretorio_dataset,'covid')
dir_target_diretorio_pneumonia_normal =r"{0}/{1}".format(dir_target_diretorio_dataset,'normal')
dir_target_diretorio_pneumonia_pneumonia =r"{0}/{1}".format(dir_target_diretorio_dataset,'pneumonia')



dir_source_covid_cohen =r'D:/Repos/datasetsGitHub/covid-chestxray-dataset'
dir_source_covid_figure1 =r'D:/Repos/datasetsGitHub/Figure1-COVID-chestxray-dataset'
dir_source_covid_Actualmed =r'D:/Repos/datasetsGitHub/Actualmed-COVID-chestxray-dataset'
dir_source_RadiographyDatabase =r'D:/Repos/datasetsGitHub/COVID-19 Radiography Database'

dir_source_rsna_pneumonia =r'D:/Repos/rsna-pneumonia-detection-challenge'






print("source")
print(dir_source_covid_cohen)
print(dir_source_covid_figure1)
print(dir_source_covid_Actualmed)
print(dir_source_RadiographyDatabase)



print("target")
print(dir_target_diretorio_raiz)
print(dir_target_diretorio_dataset)
print(dir_target_diretorio_imagem_covid)
print(dir_target_diretorio_pneumonia_normal)
print(dir_target_diretorio_pneumonia_pneumonia)






source
D:/Repos/datasetsGitHub/covid-chestxray-dataset
D:/Repos/datasetsGitHub/Figure1-COVID-chestxray-dataset
D:/Repos/datasetsGitHub/Actualmed-COVID-chestxray-dataset
D:/Repos/datasetsGitHub/COVID-19 Radiography Database
target
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/covid
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/normal
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/pneumonia


In [4]:
lista_diretorios_target=[]
lista_diretorios_target.append(dir_target_diretorio_raiz)
lista_diretorios_target.append(dir_target_diretorio_dataset),
lista_diretorios_target.append(dir_target_diretorio_imagem_covid),
lista_diretorios_target.append(dir_target_diretorio_pneumonia_normal),
lista_diretorios_target.append(dir_target_diretorio_pneumonia_pneumonia)

for item in lista_diretorios_target:
    if not os.path.exists(item):
        os.mkdir(item)
    




## metodos auxiliares

In [5]:
def clear_diretorio(diretorio):
    for file in os.listdir(diretorio):
        full_name =os.path.join(diretorio, file)
        
        if os.path.exists(full_name):
            os.remove(full_name) 
        
        

def get_imagens_covid(dataframe,diretorio):
        lista =[]
        for (i,row) in dataframe.iterrows():
                imagemName=row["filename"]
                image_source  = "{0}/{1}".format(diretorio,imagemName)
                lista.append(image_source)

        return lista


def get_imagens_diretorio(diretorio):
        imagePaths =[]
        imagePaths = list(paths.list_images(diretorio))
        imagePaths = [ item.replace("\\","/") for item in imagePaths]
        
        return imagePaths    


def move_imagens_from_source_to_target(source_lista,target):
    for source_path in source_lista:
        imagemName =source_path.split('/')[-1]
        source_target= "{0}/{1}".format(target,imagemName)
    # print(source_path)
    # print(source_target)
        if(os.path.exists(source_path)):
            if not os.path.exists(source_target):
                shutil.copy2(source_path,source_target)
    
                


## Leitura dos Datasets

In [6]:

## repositorios locais
source_dir_metadata_local_cohen  = r"{0}/{1}".format(dir_source_covid_cohen,'metadata.csv')
dir_source_imagens_Covid_cohen = r"{0}/{1}".format(dir_source_covid_cohen,'images')

source_dir_metadata_local_figure_1  = r"{0}/{1}".format(dir_source_covid_figure1,'metadata.csv')
source_dir_imagens_figure_1  = r"{0}/{1}".format(dir_source_covid_figure1,'images')

source_dir_metadata_local_Actualmed  = r"{0}/{1}".format(dir_source_covid_Actualmed,'metadata.csv')
source_dir_imagens_Actualmed = r"{0}/{1}".format(dir_source_covid_Actualmed,'images')






## diretorios pra pneumonia

dir_source_pneumonia_csv_train_detailed = r"{0}/{1}".format(dir_source_rsna_pneumonia,'stage_2_detailed_class_info.csv') 
dir_source_pneumonia_detailed_class_imagens =r"{0}/{1}".format(dir_source_rsna_pneumonia,'stage_2_train_images')



## repositorios web
metadata_web_git_cohen ='https://raw.githubusercontent.com/ieee8023/covid-chestxray-dataset/master/metadata.csv'



In [7]:
map_prognostico  =  { 'Lung Opacity': 'pneumonia',
    'Normal' :'normal',
    'COVID-19':'covid-19',
    'SARS': 'pneumonia',
    'MERS': 'pneumonia',
    'Streptococcus': 'pneumonia',
    'Klebsiella': 'pneumonia',
    'Chlamydophila': 'pneumonia',
    'Legionella': 'pneumonia',
    'Normal': 'normal',
    'Pneumonia' : 'pneumonia'

}

In [8]:
clear_diretorio(dir_target_diretorio_imagem_covid)     
clear_diretorio(dir_target_diretorio_pneumonia_pneumonia)       
clear_diretorio(dir_target_diretorio_pneumonia_normal)       

In [9]:


# df_local_cohen = pd.read_csv(source_dir_metadata_local_cohen,error_bad_lines=False)
df_local_figure_1 = pd.read_csv(source_dir_metadata_local_figure_1,error_bad_lines=False,encoding='ISO-8859-1')
df_local_cohen = pd.read_csv(source_dir_metadata_local_cohen,error_bad_lines=False,encoding='ISO-8859-1')
df_local_Actualmed = pd.read_csv(source_dir_metadata_local_Actualmed,error_bad_lines=False,encoding='ISO-8859-1')

df_local_RadiographyDatabase_Covid = pd.read_excel(r"{0}/{1}".format(dir_source_RadiographyDatabase,'COVID-19.metadata.xlsx'),error_bad_lines=False, encoding='ISO-8859-1')

df_local_RadiographyDatabase_Normal = pd.read_excel(r"{0}/{1}".format(dir_source_RadiographyDatabase,'NORMAL.metadata.xlsx'),error_bad_lines=False, encoding='ISO-8859-1')

df_local_RadiographyDatabase_Pneumonia = pd.read_excel(r"{0}/{1}".format(dir_source_RadiographyDatabase,'Viral Pneumonia.matadata.xlsx'),error_bad_lines=False, encoding='ISO-8859-1')


source_dir_metadata_RadiographyDatabase_Imagens_Covid = r"{0}/{1}".format(dir_source_RadiographyDatabase,'COVID-19')
source_dir_metadata_RadiographyDatabase_Imagens_Normal = r"{0}/{1}".format(dir_source_RadiographyDatabase,'NORMAL')
source_dir_metadata_RadiographyDatabase_Imagens_Pneumonia= r"{0}/{1}".format(dir_source_RadiographyDatabase,'Viral Pneumonia')



df_rsna = pd.read_csv(dir_source_pneumonia_csv_train_detailed,error_bad_lines=False,encoding='ISO-8859-1')

print("Quantidade registros figure_1:{0}".format(df_local_figure_1.shape))
print("Quantidade registros cohen:{0}".format(df_local_cohen.shape))
print("Quantidade registros Actualmed:{0}".format(df_local_Actualmed.shape))
print("Quantidade registros RadiographyDatabase Covid:{0}".format(df_local_RadiographyDatabase_Covid.shape))
print("Quantidade registros RadiographyDatabase Normal:{0}".format(df_local_RadiographyDatabase_Normal.shape))
print("Quantidade registros RadiographyDatabase Pneunonia:{0}".format(df_local_RadiographyDatabase_Pneumonia.shape))
print("Quantidade registros Actualmed:{0}".format(df_local_Actualmed.shape))
print("Quantidade registros Rsna:{0}".format(df_rsna.shape))

Quantidade registros figure_1:(55, 12)
Quantidade registros cohen:(682, 29)
Quantidade registros Actualmed:(238, 13)
Quantidade registros RadiographyDatabase Covid:(219, 4)
Quantidade registros RadiographyDatabase Normal:(1341, 4)
Quantidade registros RadiographyDatabase Pneunonia:(1345, 4)
Quantidade registros Actualmed:(238, 13)
Quantidade registros Rsna:(30227, 2)


## tranferencia dos arquivos de Covid 19

In [10]:
df_local_cohen['prognostico'] = df_local_cohen['finding'].map(map_prognostico)


df_local_cohen['prognostico'].value_counts()

# df_covid =  df_metadata.query('indicadorPrognostico == 1 and posicao_imagem =="PA" and tipoExame =="X-ray"')

covid-19     451
pneumonia     63
Name: prognostico, dtype: int64

In [11]:
df_local_figure_1['prognostico'] = df_local_figure_1['finding'].map(map_prognostico)


df_local_figure_1['prognostico'].value_counts()

covid-19     35
pneumonia     2
Name: prognostico, dtype: int64

In [12]:
df_local_Actualmed['prognostico'] = df_local_Actualmed['finding'].map(map_prognostico)

df_local_Actualmed['prognostico'].value_counts()



covid-19    58
Name: prognostico, dtype: int64

In [13]:

df_local_RadiographyDatabase_Covid['prognostico'] ="covid-19"


df_local_RadiographyDatabase_Covid['prognostico'].value_counts()



covid-19    219
Name: prognostico, dtype: int64

In [14]:
df_local_Actualmed['prognostico'] = df_local_Actualmed['finding'].map(map_prognostico)


df_local_Actualmed['prognostico'].value_counts()

covid-19    58
Name: prognostico, dtype: int64

In [15]:


df_rsna['prognostico'] = df_rsna['class'].map(map_prognostico)


df_rsna['prognostico'].value_counts()

pneumonia    9555
normal       8851
Name: prognostico, dtype: int64

In [16]:
lista_imagens_covid =[]
lista_imagens_pneumonia =[]
lista_imagens_normal =[]

quantidade_registros = {'normal': 0, 'pneumonia': 0, 'covid-19': 0}

# diretorio dos dados Covid

## Josephpcohen https://josephpcohen.com/w/public-covid19-dataset/

In [17]:
for row in df_local_cohen[["prognostico","filename"]].iterrows():
    imagem_name = row[1].filename
    prognostico = row[1].prognostico
    source_imagem = "{0}/{1}".format(dir_source_imagens_Covid_cohen,imagem_name)
    if os.path.exists(source_imagem):
        if(prognostico =="covid-19"):
            lista_imagens_covid.append(source_imagem)
            quantidade_registros[prognostico] +=1
        elif (prognostico =="pneumonia"):
            lista_imagens_pneumonia.append(source_imagem)
            quantidade_registros[prognostico] +=1


print(quantidade_registros)
            
            

{'normal': 0, 'pneumonia': 63, 'covid-19': 430}


In [18]:
for row in df_local_figure_1[["prognostico","patientid"]].iterrows():
    imagem_name = row[1].patientid
    prognostico = row[1].prognostico
    source_imagem = "{0}/{1}.jpg".format(source_dir_imagens_figure_1,imagem_name)
    if os.path.exists(source_imagem):
        if(prognostico =="covid-19"):
            lista_imagens_covid.append(source_imagem)
            quantidade_registros[prognostico] +=1
        elif (prognostico =="pneumonia"):
            lista_imagens_pneumonia.append(source_imagem)
            quantidade_registros[prognostico] +=1


print(quantidade_registros)

{'normal': 0, 'pneumonia': 65, 'covid-19': 460}


In [19]:
for row in df_local_Actualmed[["prognostico","imagename"]].iterrows():
    imagem_name = row[1].imagename
    prognostico = row[1].prognostico
    source_imagem = "{0}/{1}".format(source_dir_imagens_Actualmed,imagem_name)
    if os.path.exists(source_imagem):
        if(prognostico =="covid-19"):
            lista_imagens_covid.append(source_imagem)
            quantidade_registros[prognostico] +=1
        elif (prognostico =="pneumonia"):
            lista_imagens_pneumonia.append(source_imagem)
            quantidade_registros[prognostico] +=1


print(quantidade_registros)

{'normal': 0, 'pneumonia': 65, 'covid-19': 518}


In [20]:
for row in df_local_RadiographyDatabase_Covid[["prognostico","FILE NAME","FORMAT"]].iterrows():
    imagem_name ="{0}.{1}".format(row[1]["FILE NAME"],str(row[1]["FORMAT"]).lower())
    prognostico = row[1].prognostico
    source_imagem = "{0}/{1}".format(source_dir_metadata_RadiographyDatabase_Imagens_Covid,imagem_name)
    if os.path.exists(source_imagem):
        if(prognostico =="covid-19"):
            lista_imagens_covid.append(source_imagem)
            quantidade_registros[prognostico] +=1
        elif (prognostico =="pneumonia"):
            lista_imagens_pneumonia.append(source_imagem)
            quantidade_registros[prognostico] +=1

print(quantidade_registros)

{'normal': 0, 'pneumonia': 65, 'covid-19': 604}


In [33]:
df_rsna.groupby(['prognostico']).groups

{'normal': Int64Index([    3,    11,    12,    13,    21,    23,    34,    35,    39,
                45,
             ...
             30194, 30204, 30210, 30211, 30212, 30213, 30216, 30218, 30223,
             30224],
            dtype='int64', length=8851),
 'pneumonia': Int64Index([    4,     5,     8,     9,    14,    15,    16,    17,    19,
                20,
             ...
             30208, 30209, 30214, 30215, 30219, 30220, 30221, 30222, 30225,
             30226],
            dtype='int64', length=9555)}

In [39]:
for row in df_rsna[["prognostico","patientId"]].iterrows():
    imagem_name = row[1].patientId
    prognostico = row[1].prognostico
    print(row[1])
    source_imagem = "{0}/{1}.dcm".format(dir_source_pneumonia_detailed_class_imagens,imagem_name)
    if os.path.exists(source_imagem):
        dir_target =""
        if(prognostico =="normal"):
            dir_target = "{0}/{1}.jpg".format(dir_target_diretorio_pneumonia_normal,imagem_name)
        elif (prognostico =="pneumonia"):
            dir_target = "{0}/{1}.jpg".format(dir_target_diretorio_pneumonia_pneumonia,imagem_name)
        
        if not os.path.exists(dir_target):
            ds = dicom.dcmread(source_imagem)
            pixel_array_numpy = ds.pixel_array
            # cv2.imwrite(dir_target, pixel_array_numpy)
            # quantidade_registos[prognostico] +=1

prognostico                                     NaN
patientId      0004cfab-14fd-4e49-80ba-63a80b6bddd6
Name: 0, dtype: object
prognostico                                     NaN
patientId      00313ee0-9eaa-42f4-b0ab-c148ed3241cd
Name: 1, dtype: object


## tranfere as imagens do repositorio do rsna para o dataset

In [45]:


for row in df_rsna[["prognostico","patientId"]].iterrows():
    imagem_name = row[1].patientId
    prognostico = row[1].prognostico
    source_imagem = "{0}/{1}.dcm".format(dir_source_pneumonia_detailed_class_imagens,imagem_name)
    if os.path.exists(source_imagem):
        dir_target =""
        if(prognostico =="normal"):
            dir_target = "{0}/{1}.jpg".format(dir_target_diretorio_pneumonia_normal,imagem_name)
        elif (prognostico =="pneumonia"):
            dir_target = "{0}/{1}.jpg".format(dir_target_diretorio_pneumonia_pneumonia,imagem_name)
        
        if not os.path.exists(dir_target):
            ds = dicom.dcmread(source_imagem)
            pixel_array_numpy = ds.pixel_array
            cv2.imwrite(dir_target, pixel_array_numpy)
            quantidade_registos[prognostico] +=1
            



print(quantidade_registos)        


{'normal': 1, 'pneumonia': 0, 'COVID-19': 0}


In [14]:
df_covid  = pd.read_csv(source_dir_metadata_local)

In [15]:
df_covid.head(1)

,patientid,offset,sex,age,finding,survival,intubated,intubation_present,went_icu,in_icu,...,modality,date,location,folder,filename,doi,url,license,clinical_notes,other_notes
0,2,0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,X-ray,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN
